In this notebook we read in the Master XML file made using NapaTrackMater and create N, Delta times Attribute dimensional vectors. N being the number of tracks present in the chosen region, R, Delta being the chosen time interval (t_minus + t_plus) {t - t_minus, t + t_plus} and Attributes being the morphological and the dynamic properties associated with cells in the tracks. We concatenate the Attribute componenets over the chosen time interval to create a Delta times Attribute dimensional vector and create a pandas dataframe with these vectors for further analysis.

In [5]:
from napatrackmater.Trackvector import TrackVector
from pathlib import Path
from ipywidgets import interactive 

In [2]:
master_xml_path = Path('C:/Users/rando/Downloads/Mari_project/gt/master_gt_star_hyperstack.xml')
spot_csv_path = Path('C:/Users/rando/Downloads/Mari_project/gt/spots.csv')
track_csv_path = Path('C:/Users/rando/Downloads/Mari_project/gt/tracks.csv')
edges_csv_path = Path('C:/Users/rando/Downloads/Mari_project/gt/edges.csv')

In [3]:
start_time = 10
t_minus = 5
t_plus = 5
x_start = 5
x_end = 25
y_start = 5
y_end = 25
show_local_tracks = True

In [4]:
track_vectors = TrackVector(master_xml_path,spot_csv_path, track_csv_path, edges_csv_path, t_current = start_time, t_minus = t_minus, t_plus = t_plus, x_start = x_start, x_end = x_end,
                    y_start = y_start, y_end = y_end, show_tracks = show_local_tracks)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\rando\anaconda3\envs\naparienv\lib\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\rando\AppData\Local\Temp\ipykernel_37208\234999556.py", line 1, in <cell line: 1>
    interactive(track_vectors._get_track_vector_xml_data())
  File "c:\users\rando\downloads\python_workspace\napatrackmater\napatrackmater\Trackvector.py", line 68, in _get_track_vector_xml_data
AttributeError: 'TrackVector' object has no attribute 'xml_content'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\rando\anaconda3\envs\naparienv\lib\site-packages\IPython\core\interactiveshell.py", line 1982, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\rando\anaconda3\envs\naparienv\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return F